# GitPy
### https://gitpython.readthedocs.io/en/stable/index.html

### Install && Imports

In [ ]:
!pip install -r requirements.txt

In [3]:
from git import Repo

## Configure local repo

In [4]:
local_root = 'test_file_sys'

In [74]:
mrrepo = Repo(local_root)

## Display level 1 contents

In [7]:
tree = mrrepo.tree()
files_dirs = [fd for fd in tree]
files_dirs

[<git.Tree "09738889ae1914811d418aeefc52a94b50e1a3fb">,
 <git.Tree "32b408c2333033af5da5cd7eec0fbeb1e2101c59">,
 <git.Blob "e69de29bb2d1d6434b8b29ae775ad8c2e48c5391">]

#### Prettier

In [8]:
files_dirs = [(fd, fd.name, fd.type) for fd in tree]
files_dirs

[(<git.Tree "09738889ae1914811d418aeefc52a94b50e1a3fb">, 'Desktop', 'tree'),
 (<git.Tree "32b408c2333033af5da5cd7eec0fbeb1e2101c59">, 'Downloads', 'tree'),
 (<git.Blob "e69de29bb2d1d6434b8b29ae775ad8c2e48c5391">, 'file5.txt', 'blob')]

## Print all files and dirs

In [23]:
def print_files_from_git(tree, delim=' ', i = 0):
    files_dirs = [fd for fd in tree]
    for fd in files_dirs:
        print(f'{delim if i != 0 else ""}| {fd.path}, {fd.type}')
        if fd.type == "tree":
            print_files_from_git(fd, delim*4, i+1)


In [24]:
print_files_from_git(tree, '-')

| Desktop, tree
----| Desktop/1.txt, blob
----| Desktop/file, blob
| Downloads, tree
----| Downloads/dir1, tree
----------------| Downloads/dir1/file2.txt, blob
----------------| Downloads/dir1/file3.txt, blob
----| Downloads/file4.txt, blob
| file5.txt, blob


## File Versions

In [25]:
myfile = 'Downloads/dir1/file2.txt'

In [28]:
# https://stackoverflow.com/a/46984198
commits_for_file_generator = mrrepo.iter_commits('--all', max_count=100, paths=myfile)
commits_for_file = [c for c in commits_for_file_generator]
commits_for_file


[<git.Commit "480ddf7f381374b11d3e245690023b6f76f4d987">,
 <git.Commit "a8ed8bb567dff4de249781ed26cf7e6a34c74b04">,
 <git.Commit "f886763b622828dd57b01f815b10464cddcf8be6">]

In [32]:
for c in commits_for_file:
    print(str(c))

480ddf7f381374b11d3e245690023b6f76f4d987
a8ed8bb567dff4de249781ed26cf7e6a34c74b04
f886763b622828dd57b01f815b10464cddcf8be6


In [91]:
def commitIDs_for_file(repo: Repo, file: str) -> tuple:
    commits = repo.iter_commits('--all', max_count=100, paths=file)
    return tuple([str(c) for c in commits])


In [92]:
commitIDs_for_file(mrrepo, myfile)

('480ddf7f381374b11d3e245690023b6f76f4d987',
 'a8ed8bb567dff4de249781ed26cf7e6a34c74b04',
 'f886763b622828dd57b01f815b10464cddcf8be6')

#### Let get Fancy $$

In [93]:
def print_files_from_git(tree, func, repo=mrrepo, delim=' ', i = 0):
    files_dirs = [fd for fd in tree]
    for fd in files_dirs:
        if fd.type == "tree":
            print(f'{delim if i != 0 else ""}| {fd.path}, {fd.type}')
            print_files_from_git(fd, func, delim=delim*4, i=i+1)
        else:
            print(f'{delim if i != 0 else ""}| {fd.path}, {fd.type}', end=' ')
            print(func(repo, fd.path))

In [96]:
tree = mrrepo.tree()
print_files_from_git(tree, func=commitIDs_for_file, delim='-')

| Desktop, tree
----| Desktop/1.txt, blob ('ece86661a0e1925ae8ef5c4a133f2efef0ffaa30', 'f886763b622828dd57b01f815b10464cddcf8be6')
----| Desktop/file, blob ('f886763b622828dd57b01f815b10464cddcf8be6',)
| Downloads, tree
----| Downloads/dir1, tree
----------------| Downloads/dir1/file2.txt, blob ('480ddf7f381374b11d3e245690023b6f76f4d987', 'a8ed8bb567dff4de249781ed26cf7e6a34c74b04', 'f886763b622828dd57b01f815b10464cddcf8be6')
----------------| Downloads/dir1/file3.txt, blob ('f886763b622828dd57b01f815b10464cddcf8be6',)
----| Downloads/file4.txt, blob ('f886763b622828dd57b01f815b10464cddcf8be6',)
| file5.txt, blob ('f886763b622828dd57b01f815b10464cddcf8be6',)


## Printing file contents

In [117]:
myfile = 'Downloads/dir1/file2.txt'
tree = mrrepo.tree()
path = myfile.split("/")

In [118]:
sub_dir = tree
for p in path: sub_dir = sub_dir[p]
sub_dir

<git.Blob "e4d63b207dab9939a4f734aa334b92e5de0c9c04">

In [119]:
print(sub_dir.data_stream.read().decode())

random thoughts this should be my diary

monday - got a free chinese meal


### Printing previous versions

In [ ]:
myfile = 'Downloads/dir1/file2.txt'
tree = mrrepo.tree()
path = myfile.split("/")
commits = commitIDs_for_file(mrrepo, myfile)

In [125]:
# c = mrrepo.commit(commits[1])
# c_tree = c.tree
commit_trees = [mrrepo.commit(c).tree for c in commits]
targets = [commit_trees[i][f'{myfile}'] for i in range(len(commit_trees))]

In [129]:
for target in targets: print(f'{target.data_stream.read().decode()}\n', '-'*50)

random thoughts this should be my diary

monday - got a free chinese meal
 --------------------------------------------------
random thoughts this should be my diary

 --------------------------------------------------
pa$$ord
 --------------------------------------------------
